PDF to Audiobook Converter (Attractive Tkinter GUI)
---------------------------------------------------
Description:
This program lets you load any PDF file, extract its text,
convert it into spoken audio, play it, and export it as an MP3 file.

Libraries used:
- PyMuPDF (fitz): For reading PDF text
- pyttsx3: For offline text-to-speech conversion
- playsound: For simple MP3 playback
- reportlab: To generate a sample PDF (optional)

In [6]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import fitz  # PyMuPDF
import pyttsx3
from playsound import playsound
import tempfile
import os
import threading
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

In [7]:
#  Function: Extract Text
def extract_text_from_pdf(pdf_path):
    """Extract text from all pages of the given PDF."""
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    doc.close()
    return text.strip()

In [8]:
# Class for Main App
class PDFToAudioApp:
    def __init__(self, root):
        self.root = root
        self.root.title("📖 PDF to Audiobook Converter")
        self.root.geometry("900x600")
        self.root.config(bg="#f2f2f2")

        # Variables
        self.pdf_path = ""
        self.text_content = ""
        self.rate = tk.IntVar(value=150)
        self.volume = tk.DoubleVar(value=1.0)
        self.play_thread = None
        self.stop_flag = False

        # Title
        title = tk.Label(
            root, text="🎧 PDF → Audiobook Converter", font=("Helvetica", 18, "bold"), bg="#6fa3ef", fg="white", pady=10
        )
        title.pack(fill=tk.X)

        # Top Frame (Buttons)
        top_frame = tk.Frame(root, bg="#f2f2f2", pady=10)
        top_frame.pack(fill=tk.X)

        tk.Button(top_frame, text="📂 Load PDF", command=self.load_pdf, bg="#5dade2", fg="white", width=15).pack(side=tk.LEFT, padx=5)
        tk.Button(top_frame, text="📝 Generate Sample PDF", command=self.generate_sample_pdf, bg="#58d68d", fg="white", width=20).pack(side=tk.LEFT, padx=5)
        tk.Button(top_frame, text="💾 Export to MP3", command=self.export_audio, bg="#f5b041", fg="white", width=15).pack(side=tk.LEFT, padx=5)

        # Text Area (PDF text preview)
        self.text_box = tk.Text(root, wrap=tk.WORD, font=("Arial", 11), height=20)
        self.text_box.pack(fill=tk.BOTH, expand=True, padx=10, pady=5)

        # Control Frame (bottom sliders)
        control_frame = tk.Frame(root, bg="#f2f2f2", pady=10)
        control_frame.pack(fill=tk.X)

        tk.Label(control_frame, text="Rate:", bg="#f2f2f2", font=("Arial", 10, "bold")).pack(side=tk.LEFT, padx=5)
        tk.Scale(control_frame, from_=80, to=300, orient=tk.HORIZONTAL, variable=self.rate, length=150).pack(side=tk.LEFT)

        tk.Label(control_frame, text="Volume:", bg="#f2f2f2", font=("Arial", 10, "bold")).pack(side=tk.LEFT, padx=5)
        tk.Scale(control_frame, from_=0.0, to=1.0, orient=tk.HORIZONTAL, resolution=0.1, variable=self.volume, length=150).pack(side=tk.LEFT)

        # Play/Stop buttons
        tk.Button(control_frame, text="▶ Play", command=self.play_audio, bg="#52be80", fg="white", width=10).pack(side=tk.LEFT, padx=10)
        tk.Button(control_frame, text="⏹ Stop", command=self.stop_audio, bg="#e74c3c", fg="white", width=10).pack(side=tk.LEFT, padx=5)

        # Status label
        self.status = tk.Label(root, text="Status: Idle", bg="#d6eaf8", anchor="w")
        self.status.pack(fill=tk.X, padx=10, pady=3)

    #  Load PDF and Extract Text
    def load_pdf(self):
        file_path = filedialog.askopenfilename(filetypes=[("PDF Files", "*.pdf")])
        if not file_path:
            return
        try:
            self.pdf_path = file_path
            self.text_content = extract_text_from_pdf(file_path)
            if not self.text_content.strip():
                messagebox.showwarning("No text found", "This PDF may be scanned or image-only.")
            self.text_box.delete(1.0, tk.END)
            self.text_box.insert(tk.END, self.text_content)
            self.status.config(text=f"Loaded: {os.path.basename(file_path)}")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load PDF: {e}")

    #  Convert to Speech & Play
    def play_audio(self):
        if not self.text_content.strip():
            messagebox.showwarning("No text", "Please load a PDF first.")
            return

        if self.play_thread and self.play_thread.is_alive():
            messagebox.showinfo("Playing", "Audio is already playing.")
            return

        self.stop_flag = False
        self.play_thread = threading.Thread(target=self._play_worker)
        self.play_thread.start()

    def _play_worker(self):
        try:
            engine = pyttsx3.init()
            engine.setProperty("rate", self.rate.get())
            engine.setProperty("volume", self.volume.get())

            fd, tmpfile = tempfile.mkstemp(suffix=".mp3")
            os.close(fd)
            engine.save_to_file(self.text_content, tmpfile)
            engine.runAndWait()

            if not self.stop_flag:
                self.status.config(text="Playing audio...")
                playsound(tmpfile)
            self.status.config(text="Playback finished.")
            os.remove(tmpfile)
        except Exception as e:
            messagebox.showerror("Error", f"Playback failed: {e}")

    def stop_audio(self):
        """Stop playback."""
        self.stop_flag = True
        self.status.config(text="Stopped by user.")

    # Export to MP3
    def export_audio(self):
        if not self.text_content.strip():
            messagebox.showwarning("No text", "Please load a PDF first.")
            return

        save_path = filedialog.asksaveasfilename(defaultextension=".mp3", filetypes=[("MP3 Files", "*.mp3")])
        if not save_path:
            return

        try:
            engine = pyttsx3.init()
            engine.setProperty("rate", self.rate.get())
            engine.setProperty("volume", self.volume.get())
            engine.save_to_file(self.text_content, save_path)
            engine.runAndWait()
            messagebox.showinfo("Success", f"Audio saved to:\n{save_path}")
            self.status.config(text="Export complete.")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to export: {e}")

    # Generate Sample PDF
    def generate_sample_pdf(self):
        temp_path = tempfile.mktemp(suffix=".pdf")
        c = canvas.Canvas(temp_path, pagesize=letter)
        text_obj = c.beginText(50, 750)
        content = (
            "This is a sample PDF generated for testing the PDF to Audiobook Converter.\n"
            "It demonstrates reading and converting text into audio.\n\n"
        )
        for _ in range(3):
            text_obj.textLines(content * 3)
            c.drawText(text_obj)
            c.showPage()
            text_obj = c.beginText(50, 750)
        c.save()

        self.pdf_path = temp_path
        self.text_content = extract_text_from_pdf(temp_path)
        self.text_box.delete(1.0, tk.END)
        self.text_box.insert(tk.END, self.text_content)
        self.status.config(text="Sample PDF generated and loaded.")

In [9]:
# Run the Application
if __name__ == "__main__":
    root = tk.Tk()
    app = PDFToAudioApp(root)
    root.mainloop()
